In [1]:
class CFG:
    exp = '018'
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + CFG.exp
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [2]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [3]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 3.8 MB/s 
CPU times: user 1.85 s, sys: 407 ms, total: 2.26 s
Wall time: 27.3 s


In [4]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [5]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{CFG.exp}')
output_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/stacking{CFG.exp}')
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [6]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
print(get_score(oof_df['score'], oof_df['pred']))
oof_df.head()

0.8280995878197323


,id,anchor,target,context,score,context_text,text,text_rev,fold,pred
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.403401
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.639036
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.230556
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.412804
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000497


In [8]:
tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')

In [9]:
train = oof_df.copy()

# 特徴量エンジニアリング
def tok_len(sentence):
    return len(tokenizer.tokenize(sentence))

def feature_engineering(df):
    df['section'] = df['context'].map(lambda s: s[0])

    df['anchor_tok_len'] = df['anchor'].map(tok_len)
    df['target_tok_len'] = df['target'].map(tok_len)
    df['context_tok_len'] = df['context_text'].map(tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    return df


train = feature_engineering(train)
display(train.drop(columns=['context_text']).head())

,id,anchor,target,context,score,text,text_rev,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,abatement. [SEP] abatement of pollution. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,abatement of pollution. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.403401,A,1,3,20,28,-2,0.333333,-19,0.05,-17,0.15
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,abatement. [SEP] act of abating. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,act of abating. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.639036,A,1,5,20,30,-4,0.200000,-19,0.05,-15,0.25
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,abatement. [SEP] active catalyst. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,active catalyst. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.230556,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,abatement. [SEP] eliminating process. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,eliminating process. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.412804,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,abatement. [SEP] forest region. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,forest region. [SEP] abatement. [SEP] HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL.,0,0.000497,A,1,2,20,27,-1,0.500000,-19,0.05,-18,0.10


In [10]:
# カテゴリ変数の変換
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [11]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'pred', 'section', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded'], dtype='object')


In [12]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['pred', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded']


In [13]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [14]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [ ]:
best_params = tuning_params(train, params)
best_params

[I 2022-06-08 15:52:42,129] A new study created in memory with name: no-name-b0795106-b807-4f1c-ae61-6c43abb86bde


train: 27535, val: 8938


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l2: 0.0652884	valid_1's l2: 0.0674474
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 0.0651902	valid_1's l2: 0.0673385
[3]	valid_0's l2: 0.0642988	valid_1's l2: 0.0664157
[4]	valid_0's l2: 0.0634251	valid_1's l2: 0.0655116
[5]	valid_0's l2: 0.0625687	valid_1's l2: 0.064625
[6]	valid_0's l2: 0.0617294	valid_1's l2: 0.0637563
[7]	valid_0's l2: 0.0609069	valid_1's l2: 0.0629041
[8]	valid_0's l2: 0.0608189	valid_1's l2: 0.0628066
[9]	valid_0's l2: 0.0600143	valid_1's l2: 0.0619734
[10]	valid_0's l2: 0.0592259	valid_1's l2: 0.0611572
[11]	valid_0's l2: 0.0584529	valid_1's l2: 0.0603568
[12]	valid_0's l2: 0.0576957	valid_1's l2: 0.059573
[13]	valid_0's l2: 0.0569532	valid_1's l2: 0.058804
[14]	valid_0's l2: 0.0562257	valid_1's l2: 0.0580501
[15]	valid_0's l2: 0.0555127	valid_1's l2: 0.0573111
[16]	valid_0's l2: 0.0548136	valid_1's l2: 0.0565863
[17]	valid_0's l2: 0.0541286	valid_1's l2: 0.055877
[18]	valid_0's l2: 0.0534568	valid_1's l2: 0.05518

feature_fraction, val_score: 0.020850:  14%|#4        | 1/7 [00:05<00:34,  5.70s/it]

...

[1]	valid_0's l2: 0.0652692	valid_1's l2: 0.0674428
[2]	valid_0's l2: 0.0651499	valid_1's l2: 0.067327
[3]	valid_0's l2: 0.0642399	valid_1's l2: 0.0663991
[4]	valid_0's l2: 0.0633478	valid_1's l2: 0.0654903
[5]	valid_0's l2: 0.0624727	valid_1's l2: 0.0646
[6]	valid_0's l2: 0.0616157	valid_1's l2: 0.0637273
[7]	valid_0's l2: 0.060775	valid_1's l2: 0.0628724
[8]	valid_0's l2: 0.0606678	valid_1's l2: 0.0627689
[9]	valid_0's l2: 0.0598462	valid_1's l2: 0.0619321
[10]	valid_0's l2: 0.0590397	valid_1's l2: 0.0611147
[11]	valid_0's l2: 0.0582499	valid_1's l2: 0.0603126
[12]	valid_0's l2: 0.0574752	valid_1's l2: 0.0595272
[13]	valid_0's l2: 0.0567164	valid_1's l2: 0.0587555
[14]	valid_0's l2: 0.0559728	valid_1's l2: 0.0579994
[15]	valid_0's l2: 0.0552435	valid_1's l2: 0.0572596
[16]	valid_0's l2: 0.054529	valid_1's l2: 0.0565329
[17]	valid_0's l2: 0.0538285	valid_1's l2: 0.0558224
[18]	valid_0's l2: 0.0531418	valid_1's l2: 0.055124
[19]	valid_0's l2: 0.0530541	valid_1's l2: 0.0550404
[20]	vali

min_data_in_leaf, val_score: 0.020790:  60%|######    | 3/5 [00:25<00:17,  8.53s/it]

[1]	valid_0's l2: 0.0652728	valid_1's l2: 0.0674454
[2]	valid_0's l2: 0.065159	valid_1's l2: 0.067331
[3]	valid_0's l2: 0.0642512	valid_1's l2: 0.0664045
[4]	valid_0's l2: 0.0633615	valid_1's l2: 0.0654971
[5]	valid_0's l2: 0.0624895	valid_1's l2: 0.0646069
[6]	valid_0's l2: 0.0616348	valid_1's l2: 0.0637323
[7]	valid_0's l2: 0.0607968	valid_1's l2: 0.0628787
[8]	valid_0's l2: 0.0606953	valid_1's l2: 0.0627774
[9]	valid_0's l2: 0.0598758	valid_1's l2: 0.0619424
[10]	valid_0's l2: 0.059072	valid_1's l2: 0.0611251
[11]	valid_0's l2: 0.0582843	valid_1's l2: 0.0603235
[12]	valid_0's l2: 0.0575123	valid_1's l2: 0.0595383
[13]	valid_0's l2: 0.0567555	valid_1's l2: 0.0587684
[14]	valid_0's l2: 0.0560139	valid_1's l2: 0.0580143
[15]	valid_0's l2: 0.0552868	valid_1's l2: 0.0572752
[16]	valid_0's l2: 0.0545741	valid_1's l2: 0.0565504
[17]	valid_0's l2: 0.0538753	valid_1's l2: 0.0558401
[18]	valid_0's l2: 0.053191	valid_1's l2: 0.0551437
[19]	valid_0's l2: 0.0531082	valid_1's l2: 0.0550611
[20]	v

min_data_in_leaf, val_score: 0.020790:  80%|########  | 4/5 [00:35<00:09,  9.04s/it]

[1]	valid_0's l2: 0.0652777	valid_1's l2: 0.0674423
[2]	valid_0's l2: 0.0651692	valid_1's l2: 0.0673298
[3]	valid_0's l2: 0.064268	valid_1's l2: 0.0664022
[4]	valid_0's l2: 0.0633845	valid_1's l2: 0.0654933
[5]	valid_0's l2: 0.0625185	valid_1's l2: 0.0646027
[6]	valid_0's l2: 0.0616703	valid_1's l2: 0.0637304
[7]	valid_0's l2: 0.0608382	valid_1's l2: 0.0628748
[8]	valid_0's l2: 0.0607414	valid_1's l2: 0.0627745
[9]	valid_0's l2: 0.0599277	valid_1's l2: 0.0619393
[10]	valid_0's l2: 0.0591301	valid_1's l2: 0.0611194
[11]	valid_0's l2: 0.0583485	valid_1's l2: 0.0603161
[12]	valid_0's l2: 0.0575821	valid_1's l2: 0.059528
[13]	valid_0's l2: 0.0568307	valid_1's l2: 0.0587563
[14]	valid_0's l2: 0.0560946	valid_1's l2: 0.0579992
[15]	valid_0's l2: 0.0553724	valid_1's l2: 0.0572567
[16]	valid_0's l2: 0.0546654	valid_1's l2: 0.0565303
[17]	valid_0's l2: 0.0539722	valid_1's l2: 0.0558173
[18]	valid_0's l2: 0.0532926	valid_1's l2: 0.0551199
[19]	valid_0's l2: 0.0532134	valid_1's l2: 0.0550382
[20]

In [ ]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [ ]:
pred = train_fn(train, best_params)

Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0268277	valid_1's l2: 0.0266575
[200]	training's l2: 0.0193769	valid_1's l2: 0.0205927
[300]	training's l2: 0.0175911	valid_1's l2: 0.0200143
Early stopping, best iteration is:
[294]	training's l2: 0.0176607	valid_1's l2: 0.0200122
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0263689	valid_1's l2: 0.0277965
[200]	training's l2: 0.0189014	valid_1's l2: 0.0221791
[300]	training's l2: 0.0171368	valid_1's l2: 0.0217457
Early stopping, best iteration is:
[278]	training's l2: 0.0173958	valid_1's l2: 0.0217304
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 0.0266568	valid_1's l2: 0.0290025
[200]	training's l2: 0.0192426	valid_1's l2: 0.0221724
[300]	training's l2: 0.0174836	valid_1's l2: 0.021138
[400]	training's l2: 0.0166943	valid_1's l2: 0.0210919
Early stopping, best iteration is:
[357]	training's l2: 0.0169874	valid_1's l2: 0.021

In [ ]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
display(importance_df.sort_values('importance', ascending=False))

,feature,importance
0,pred,63410.575142
10,len_tgt_cnt_div,682.465205
8,len_anc_cnt_div,651.612224
4,input_len,581.268534
2,target_tok_len,560.695663
3,context_tok_len,395.891068
9,len_tgt_cnt_diff,360.076883
11,section_encoded,325.393192
7,len_anc_cnt_diff,319.600850
5,len_anc_tgt_diff,279.627947


In [ ]:
# optunaのbest params
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred[['id', 'score', 'pred', 'pred_lgb']].head()

0.8280995878197323
0.8241152357318047


,id,score,pred,pred_lgb
0,37d61fd2272659b1,0.50,0.403401,0.433267
1,7b9652b17b68b7a4,0.75,0.639036,0.615205
2,36d72442aefd8232,0.25,0.230556,0.248800
3,5296b0c19e1ce60e,0.50,0.412804,0.420865
4,54c1e3b9184cb5b6,0.00,0.000497,0.043843


In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()

Starting upload for file section_encoder.pkl
100% 323/323 [00:04<00:00, 73.7B/s]
Upload successful: section_encoder.pkl (323B)
Starting upload for file cols.pkl
100% 256/256 [00:04<00:00, 59.8B/s]
Upload successful: cols.pkl (256B)
Starting upload for file lgb_fold0.txt
100% 6.03M/6.03M [00:03<00:00, 1.84MB/s]
Upload successful: lgb_fold0.txt (6MB)
Starting upload for file lgb_fold1.txt
100% 5.70M/5.70M [00:04<00:00, 1.22MB/s]
Upload successful: lgb_fold1.txt (6MB)
Starting upload for file lgb_fold2.txt
100% 7.33M/7.33M [00:05<00:00, 1.45MB/s]
Upload successful: lgb_fold2.txt (7MB)
Starting upload for file lgb_fold3.txt
100% 6.48M/6.48M [00:05<00:00, 1.35MB/s]
Upload successful: lgb_fold3.txt (6MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/hanejiyuto/PPPM-stacking-exp018
